In [ ]:
# Traffic Congestion Prediction
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import os



In [ ]:
CSV_PATH = r"E:\citysense360\data\traffic\traffic.csv"
TIME_STEPS = 24        
BATCH_SIZE = 32
EPOCHS = 30
MODEL_SAVE_PATH = r"E:\citysense360\models\traffic_lstm.h5"

df = pd.read_csv(CSV_PATH)
df.head()


In [ ]:
df["date_time"] = pd.to_datetime(
    df["date_time"],
    dayfirst=True
)
df["date_time"] = pd.to_datetime(df["date_time"])
df = df.sort_values("date_time")

traffic_values = df[["traffic_volume"]].values

print("Total records:", len(traffic_values))



In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
traffic_scaled = scaler.fit_transform(traffic_values)

generator = TimeseriesGenerator(
    data=traffic_scaled,
    targets=traffic_scaled,
    length=TIME_STEPS,
    batch_size=BATCH_SIZE
)

X_sample, y_sample = generator[0]
print("X shape:", X_sample.shape)  
print("y shape:", y_sample.shape)  



In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(
        units=64,
        activation="relu",
        input_shape=(TIME_STEPS, 1)
    ),
    Dropout(0.2),
    Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.summary()



In [ ]:
history = model.fit(
    generator,
    epochs=EPOCHS,
    verbose=1
)


In [ ]:
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)
model.save(MODEL_SAVE_PATH)

print(f"Model saved at: {MODEL_SAVE_PATH}")
